In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163690 sha256=8b1604603048862a5bdceadcc88e6cfcae1361f8fa24378b5e9882f84f9dfb90
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import surprise
import os

In [ ]:
os.chdir("/content/drive/MyDrive/Cases/filmtrust")
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [5]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [6]:
similarity_options = {'name': 'cosine', 'user_based': True}
# Default k = 40
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


Expected rating for user 50 for item 217:

In [7]:
pred = algo.predict(uid='50',iid='217')
print(pred.est)

3.0028030537791928


In [8]:
pred

Prediction(uid='50', iid='217', r_ui=None, est=3.0028030537791928, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

Total Items:

In [9]:
iids = ratings['iid'].unique()
print(iids)

[   1    2    3 ... 2069 2070 2071]


The list of items rated by user 50:

In [10]:
u_iid = ratings[ratings['uid']==50]['iid'].unique()
print(u_iid)

[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


In [11]:
len(u_iid)

39

List of the items not rated by user 50:

In [12]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[  14   15   16 ... 2069 2070 2071]


In [13]:
len(iids_to_predict)

2032

Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
testset

In [16]:
predictions = algo.test(testset)

In [17]:
type(predictions), len(predictions)

(list, 2032)

In [18]:
predictions[:5]

[Prediction(uid=50, iid=14, r_ui=0.0, est=1.0249112823172175, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=50, iid=15, r_ui=0.0, est=2.3010819030660024, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=50, iid=16, r_ui=0.0, est=3.365656247496976, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=50, iid=18, r_ui=0.0, est=3.4750887176827825, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=50, iid=19, r_ui=0.0, est=2.9501774353655654, details={'actual_k': 2, 'was_impossible': False})]

Getting the item with highest expected rating

In [19]:
pred_ratings = [ (predictions[i].iid,predictions[i].est) for i in range(0,len(predictions)) ]
predicted_rating = pd.DataFrame(pred_ratings, columns=['iid','est_rating'])
predicted_rating = pd.DataFrame(pred_ratings, columns=['iid','est_rating'])
predicted_rating.sort_values(by='est_rating', ascending=False)

,iid,est_rating
562,602,4.0
1750,1790,4.0
563,603,4.0
561,601,4.0
1753,1793,4.0
...,...,...
1480,1520,0.5
1487,1527,0.5
895,935,0.5
1514,1554,0.5


Tuning for best K

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

### User-Based Filtering

In [ ]:
param_grid = {'k': np.arange(30,70,10),  'user_based':[True]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [True]}

In [ ]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8641633357915124


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'k': 60, 'user_based': True}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.

### Item-Based Filtering

In [ ]:
param_grid = {'k': np.arange(30,70,10), 'user_based':[False]}
param_grid

{'k': array([30, 40, 50, 60]), 'user_based': [False]}

In [ ]:
kfold = KFold(n_splits=5, random_state=23, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8641633357915124


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'k': 60, 'user_based': False}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


The recommendations can be generated for any user with the object **algo**.